#  Газетный корпус


Для начала, необходимо выкачать статьи с сайта газеты
В моем случае, это газета ["Уездные вести"](http://smi67.ru/)


In [1]:
import re
import urllib.request
import csv

def get_links(pageUrl):
    try:
        page = urllib.request.urlopen(pageUrl)  # Считывает код страницы
        text = page.read().decode('utf-8')
    except:
        print('Error at', pageUrl)
    articles_links = re.findall(r'class="entry-title"><a href="(.*?)" rel="bookmark">', text)  # Ищет на странице ссылки на статьи
    return articles_links

Следующая функция очень сложная, но я объясню

In [ ]:
def download_page(pageUrl, path):
    try:
        page = urllib.request.urlopen(pageUrl)
        text = page.read().decode('utf-8')

        regPostcontent_1 = re.compile('<p style="text-align: justify;">.*?<hr>', flags=re.DOTALL) #Регулярка для поиска текста
        regPostcontent_2 = re.compile('<p style="text-align: justify;">.*?<hr />', flags=re.DOTALL) #Второй вариант,встречается в некоторых статьях
        content = regPostcontent_1.findall(text)
        if len(content) == 0:
            content = regPostcontent_2.findall(text) 

        if len(content) > 0: #Проверка, действительно ли нашелся текст
            new_content = []
            regTag2 = re.compile('<.*?>', re.DOTALL)
            regSpace2 = re.compile('\s{2,}', re.DOTALL)
            regDescr2 = re.compile('\n', re.DOTALL)
            regSimbols = re.compile('&#\d*?;', re.DOTALL)
            for c in content: #Чистим текст от ненужных знаков и тэгов
                clean_c = regSpace2.sub("", c)
                clean_c = regTag2.sub("", clean_c)
                clean_c = regDescr2.sub("", clean_c)
                clean_c = regSimbols.sub("", clean_c)
                new_content.append(clean_c)
            for c in new_content:
                c_clean = c.replace("&nbsp;", " ")

            new_title = []
            titles = re.findall('<h1 class="entry-title">(.*?)</h1>', text) #Поиск заголовка
            for t in titles: #Иногда попадаются лишние символы, поэтому почистим
                clean_t = regSpace2.sub("", t)
                clean_t = regTag2.sub("", clean_t)
                clean_t = regDescr2.sub("", clean_t)
                clean_t = regSimbols.sub("", clean_t)
                new_title.append(clean_t)

            author = re.findall(r'<span class="author vcard"><a class=".*?" href="http://smi67.ru/author/.*?">(.*?)</a></span>', text) #Поиск автора
            data_time = re.findall(r'<time class="entry-date published" datetime=".*?">(.*?)</time>', text)
            if len(data_time) == 0: 
                data_time = re.findall(r'<time class="entry-date published updated" datetime=".*?">(.*?)</time>', text) #Второй вариант для поиска даты

            all = '@au' + ' ' + author[0] + '\n' + '@ti' + ' ' + new_title[0] + '\n' + '@da' + ' ' + data_time[0] + '\n' + '@url' + ' ' + pageUrl + '\n' +c_clean

            with open(path, 'w', encoding='utf-8') as f:
                f.write(all) 
            with open('newspaper\metadata.csv', mode='a', encoding="utf-8") as csv_file:
                fieldnames = ['path', 'author', 'sex', 'birthday', 'header', 'created', 'sphere', 'genre_fi', 'type',
                              'topic', 'chronotop', 'style', 'audience_age', 'audience_level', 'audience_size',
                              'source', 'publication', 'publisher', 'publ_year', 'medium', 'country', 'region',
                              'language']

                #writer.writeheader() - Для первого запуска, для написания названий колонок
                writer = csv.DictWriter(csv_file, fieldnames=fieldnames, delimiter='\t')

                writer.writerow({'author': author[0],
                                 'header': titles[0],
                                 'created': data_time[0],
                                 'sphere': 'публицистика',
                                 'style': 'нейтральный',
                                 'audience_age': 'н-возраст',
                                 'audience_level': 'н-уровень',
                                 'audience_size': 'районная',
                                 'source': pageUrl,
                                 'publication': 'Уездные вести'})
    except:
        print('Error at', pageUrl)
    return


In [ ]:
def main():
    commonUrl = 'http://smi67.ru/date/'
    months = ['01/', '02/', '03/', '04/', '05/', '06/', '07/', '08/', '09/', '10/'] #Номера месяцев

    all_links_2018 = []
    all_links_2017 = []
    year = ['2018', '2017']
    for y in year: #Статьи за год
        for m in months: #Статьи за месяц
            pageUrl_raw = commonUrl + y +'/' + m # За каждый месяц - несколько страницы со статьями
            for i in range(2, 7):
                pageUrl = pageUrl_raw + 'page/' + str(i) + '/'   
                for links in get_links(pageUrl):
                    if y == '2018':
                        all_links_2018.append(links)
                    else:
                        all_links_2017.append(links)
    m = 0
    for links in all_links_2018: #Статьи за 2018 год
        path_2018 = 'newspaper\plain\2018\article_' + str(m) +'.txt' #Путь к статьям за 2018 год
        #print(path_2018)
        download_page(links, path_2018)
        m = m + 1
    n = 0
    for links in all_links_2017: #Статьи за 2017 год
        path_2017 = 'newspaper\plain\2017\article_' + str(n) +'.txt' #Путь к статьям за 2018 год
        #print(path_2017)
        download_page(links, path_2017)
        n = n + 1

if __name__ == '__main__':
    main()

Следующая часть программы нацелена на mystem

In [ ]:
import os
articles_2018_list = os.listdir(r'C:\mystem-3.1-win-64bit\newspaper\plain\2018')
articles_2017_list = os.listdir(r'C:\mystem-3.1-win-64bit\newspaper\plain\2017')
for article_name in articles_2018_list:
    raw_path_1 = "C:\mystem-3.1-win-64bit\mystem.exe --eng-gr -nlcgid %s %s"
    raw_path_2 = "C:\mystem-3.1-win-64bit\mystem.exe --eng-gr -nlcgid --format xml %s %s"
    path_plain_articles_1 = raw_path_1 % (r'C:\mystem-3.1-win-64bit\newspaper\plain\2018' + '\\' + article_name, r'C:\mystem-3.1-win-64bit\newspaper\mystem-plain\2018' + '\\' + article_name)
    path_plain_articles_2 = raw_path_2 % (r'C:\mystem-3.1-win-64bit\newspaper\plain\2018' + '\\' + article_name, r'C:\mystem-3.1-win-64bit\newspaper\mystem-xml\2018' + '\\' + article_name.replace('txt', 'xml'))
    print(path_plain_articles_1)
    print(path_plain_articles_2)
    run_mystem = os.system(path_plain_articles_1)
    run_mustem_xml = os.system(path_plain_articles_2)
for article_name in articles_2017_list:
    raw_path_1 = "C:\mystem-3.1-win-64bit\mystem.exe --eng-gr -nlcgid %s %s"
    raw_path_2 = "C:\mystem-3.1-win-64bit\mystem.exe --eng-gr -nlcgid --format xml %s %s"
    path_plain_articles_1 = raw_path_1 % (r'C:\mystem-3.1-win-64bit\newspaper\plain\2017' + '\\' + article_name, r'C:\mystem-3.1-win-64bit\newspaper\mystem-plain\2017' + '\\' + article_name)
    path_plain_articles_2 = raw_path_2 % (r'C:\mystem-3.1-win-64bit\newspaper\plain\2017' + '\\' + article_name, r'C:\mystem-3.1-win-64bit\newspaper\mystem-xml\2017' + '\\' + article_name.replace('txt', 'xml'))
    run_mystem = os.system(path_plain_articles_1)
    run_mustem_xml = os.system(path_plain_articles_2)